In [1]:
#| default_exp relationships

In [2]:
#| export
import jax
import jax.numpy as jnp
from jax import lax
import numpy as np
from functools import partial

# Import Population for type hinting and data access
from chewc.population import Population

@jax.jit
def calc_g_matrix(geno_dosage: jnp.ndarray) -> jnp.ndarray:
    """
    Calculate genomic relationship matrix (G-matrix) using VanRaden Method 1.
    
    Args:
        geno_dosage: A (n_ind, n_markers) array of dosages.

    Returns:
        The (n_ind, n_ind) genomic relationship matrix.
    """
    n_ind, n_markers = geno_dosage.shape
    
    # Calculate allele frequencies (p) for each marker
    p = jnp.mean(geno_dosage, axis=0) / 2.0
    
    # Center the genotype matrix M
    P = 2 * p
    M = geno_dosage - P
    
    # Calculate the denominator: 2 * sum(p * (1 - p))
    denominator = 2 * jnp.sum(p * (1 - p))
    
    # Calculate G = (M M') / denominator
    G = (M @ M.T) / denominator
    
    # Add a small regularization term for numerical stability
    epsilon = 1e-6
    return G + jnp.identity(n_ind) * epsilon


@jax.jit
def calc_a_inverse_matrix_pedigree_jax(pop: Population) -> jnp.ndarray:
    """
    Calculates the inverse of the pedigree-based A-matrix directly using
    Henderson's rules in a JIT-compilable JAX function.

    Args:
        pop: The Population object containing pedigree information.

    Returns:
        A JAX array representing the A-inverse matrix.
    """
    n_ind = pop.nInd
    mother_iids = jnp.asarray(pop.mother)
    father_iids = jnp.asarray(pop.father)
    
    # This dictionary is created outside the loop and is compatible with JIT
    id_to_iid = {int(pub_id): int(iid) for pub_id, iid in zip(pop.id, pop.iid)}
    unknown_parent_iid = -1

    def loop_body(i, A_inv):
        sire_pub_id = father_iids[i]
        dam_pub_id = mother_iids[i]
        
        s = id_to_iid.get(int(sire_pub_id), unknown_parent_iid)
        d = id_to_iid.get(int(dam_pub_id), unknown_parent_iid)

        # Determine the case based on parent availability for lax.switch
        case_index = (s != unknown_parent_iid) + (d != unknown_parent_iid) * 2
        
        # Define update functions for each of the four parental cases
        def case_0_both_unknown(mat):
            return mat.at[i, i].add(1.0)

        def case_1_sire_known(mat):
            d_ii = 1.0 # Assumes non-inbred parents
            val = 1.0 / d_ii
            return mat.at[i, i].add(val).at[s, s].add(0.25 * val).at[i, s].add(-0.5 * val).at[s, i].add(-0.5 * val)

        def case_2_dam_known(mat):
            d_ii = 1.0 # Assumes non-inbred parents
            val = 1.0 / d_ii
            return mat.at[i, i].add(val).at[d, d].add(0.25 * val).at[i, d].add(-0.5 * val).at[d, i].add(-0.5 * val)

        def case_3_both_known(mat):
            d_ii = 1.0 - 0.25 * (mat[s, s] - 1.0 + mat[d, d] - 1.0) # Placeholder for inbreeding
            val = 1.0 / d_ii
            return (mat.at[i, i].add(val)
                       .at[s, s].add(0.25 * val)
                       .at[d, d].add(0.25 * val)
                       .at[s, d].add(0.25 * val)
                       .at[d, s].add(0.25 * val)
                       .at[i, s].add(-0.5 * val)
                       .at[s, i].add(-0.5 * val)
                       .at[i, d].add(-0.5 * val)
                       .at[d, i].add(-0.5 * val))

        return lax.switch(case_index, [case_0_both_unknown, case_1_sire_known, case_2_dam_known, case_3_both_known], A_inv)

    # Initialize A-inverse matrix and loop through individuals
    initial_A_inv = jnp.zeros((n_ind, n_ind))
    A_inv = lax.fori_loop(0, n_ind, loop_body, initial_A_inv)
    return A_inv

#| export

"""
Relationship matrix calculations for chewc genetic simulations.
Supports both genomic (G-matrix) and IBD-based (A-matrix) calculations.
"""

import jax
import jax.numpy as jnp
from functools import partial
from chewc.population import Population

@jax.jit
def calc_ibd_matrix(pop_ibd: jnp.ndarray) -> jnp.ndarray:
    """
    Calculate IBD-based relationship matrix from founder haplotype IDs.
    
    This is the "true" additive relationship matrix based on actual 
    identity by descent, not just pedigree relationships.
    
    Args:
        pop_ibd: IBD array of shape (nInd, nChr, ploidy, nLoci)
        
    Returns:
        Symmetric relationship matrix of shape (nInd, nInd)
    """
    n_ind, n_chr, ploidy, n_loci = pop_ibd.shape
    
    # Reshape for easier comparison: (nInd, total_haploid_loci)
    ibd_flat = pop_ibd.reshape(n_ind, n_chr * ploidy * n_loci)
    
    # For each pair of individuals, count shared founder alleles
    # This is vectorized using broadcasting
    def calc_relationship_pair(ibd_i, ibd_j):
        # ibd_i and ibd_j are (total_haploid_loci,) arrays
        # Reshape to (ploidy, chr*loci) for proper comparison
        ibd_i_reshaped = ibd_i.reshape(ploidy, -1)  # (ploidy, chr*loci)
        ibd_j_reshaped = ibd_j.reshape(ploidy, -1)
        
        # Count matches between all haplotype pairs
        matches = 0
        for h1 in range(ploidy):
            for h2 in range(ploidy):
                matches += jnp.sum(ibd_i_reshaped[h1] == ibd_j_reshaped[h2])
        
        # Relationship coefficient: shared alleles / total possible
        total_comparisons = ploidy * ploidy * (n_chr * n_loci)
        return matches / total_comparisons
    
    # Vectorize the pairwise calculation
    vmap_calc = jax.vmap(
        jax.vmap(calc_relationship_pair, in_axes=(None, 0)), 
        in_axes=(0, None)
    )
    
    # Calculate full matrix
    ibd_matrix = vmap_calc(ibd_flat, ibd_flat)
    
    # Ensure symmetry (should already be symmetric)
    return (ibd_matrix + ibd_matrix.T) / 2

@jax.jit  
def calc_g_matrix(geno_dosage: jnp.ndarray) -> jnp.ndarray:
    """
    Calculate genomic relationship matrix (moved from predict.py).
    VanRaden Method 1.
    """
    n_ind, n_markers = geno_dosage.shape
    
    # Calculate allele frequencies
    p = jnp.mean(geno_dosage, axis=0) / 2.0
    
    # Center genotypes  
    P = 2 * p
    Z = geno_dosage - P
    
    # Calculate G matrix
    denominator = 2 * jnp.sum(p * (1 - p))
    G = (Z @ Z.T) / denominator
    
    # Add regularization
    epsilon = 1e-6
    return G + jnp.identity(n_ind) * epsilon

def calc_a_matrix_pedigree(pop: Population) -> jnp.ndarray:
    """Calculate pedigree-based A-matrix with proper ID mapping."""
    n_ind = pop.nInd
    A = np.zeros((n_ind, n_ind))
    
    # Create mapping from public ID to internal index
    id_to_iid = {int(pub_id): int(iid) for pub_id, iid in zip(pop.id, pop.iid)}
    mother_ids = np.array(pop.mother)
    father_ids = np.array(pop.father)
    
    # Henderson's algorithm
    for i in range(n_ind):
        A[i, i] = 1.0
        
        dam_pub_id = mother_ids[i] if mother_ids[i] >= 0 else None
        sire_pub_id = father_ids[i] if father_ids[i] >= 0 else None
        
        dam_idx = id_to_iid.get(dam_pub_id) if dam_pub_id is not None else None
        sire_idx = id_to_iid.get(sire_pub_id) if sire_pub_id is not None else None
        
        if dam_idx is not None and sire_idx is not None:
            A[i, i] = 1.0 + 0.5 * A[dam_idx, sire_idx]
            for j in range(i):
                A[i, j] = A[j, i] = 0.5 * (A[j, dam_idx] + A[j, sire_idx])
        elif dam_idx is not None:
            for j in range(i):
                A[i, j] = A[j, i] = 0.5 * A[j, dam_idx]
        elif sire_idx is not None:
            for j in range(i):
                A[i, j] = A[j, i] = 0.5 * A[j, sire_idx]
    
    return jnp.array(A)


import jax
import jax.numpy as jnp
from jax import lax
# Assuming 'Population' is a class or dataclass available in your environment
# from your_library import Population

def calc_a_inverse_matrix_pedigree_jax(pop) -> jnp.ndarray:
    """
    Calculates the inverse of the pedigree-based A-matrix directly using
    Henderson's rules. This version runs in regular Python, not JIT.
    """
    n_ind = pop.nInd
    mother_iids = jnp.asarray(pop.mother)
    father_iids = jnp.asarray(pop.father)

    # This dictionary creation is the reason JIT fails. It needs concrete values.
    id_to_iid = {int(pub_id): int(iid) for pub_id, iid in zip(pop.id, pop.iid)}
    unknown_parent_iid = -1

    A_inv = jnp.zeros((n_ind, n_ind))

    # We use a standard Python loop here.
    for i in range(n_ind):
        sire_pub_id = father_iids[i]
        dam_pub_id = mother_iids[i]

        s = id_to_iid.get(int(sire_pub_id), unknown_parent_iid)
        d = id_to_iid.get(int(dam_pub_id), unknown_parent_iid)

        # Henderson's rules applied based on known parents
        if s == unknown_parent_iid and d == unknown_parent_iid:
            # Both parents unknown
            A_inv = A_inv.at[i, i].add(1.0)
        elif s != unknown_parent_iid and d == unknown_parent_iid:
            # Sire known, dam unknown
            A_inv = A_inv.at[i, i].add(4/3)
            A_inv = A_inv.at[s, s].add(1/3)
            A_inv = A_inv.at[i, s].add(-2/3)
            A_inv = A_inv.at[s, i].add(-2/3)
        elif s == unknown_parent_iid and d != unknown_parent_iid:
            # Dam known, sire unknown
            A_inv = A_inv.at[i, i].add(4/3)
            A_inv = A_inv.at[d, d].add(1/3)
            A_inv = A_inv.at[i, d].add(-2/3)
            A_inv = A_inv.at[d, i].add(-2/3)
        else:
            # Both parents known
            A_inv = A_inv.at[i, i].add(2.0)
            A_inv = A_inv.at[s, s].add(0.5)
            A_inv = A_inv.at[d, d].add(0.5)
            A_inv = A_inv.at[s, d].add(0.5)
            A_inv = A_inv.at[d, s].add(0.5)
            A_inv = A_inv.at[i, s].add(-1.0)
            A_inv = A_inv.at[s, i].add(-1.0)
            A_inv = A_inv.at[i, d].add(-1.0)
            A_inv = A_inv.at[d, i].add(-1.0)
            
    return A_inv

# Convenience function for the Population class
def calc_relationship_matrices(pop: Population, method: str = "ibd") -> jnp.ndarray:
    """
    Calculate relationship matrix using specified method.
    
    Args:
        pop: Population object
        method: "ibd", "genomic", or "pedigree"
        
    Returns:
        Relationship matrix
    """
    if method == "ibd":
        if pop.ibd is None:
            raise ValueError("IBD data not available. Use 'genomic' or 'pedigree' method.")
        return calc_ibd_matrix(pop.ibd)
    
    elif method == "genomic":
        return calc_g_matrix(pop.dosage)
    
    elif method == "pedigree":
        return calc_a_matrix_pedigree(pop)
    
    else:
        raise ValueError(f"Unknown method: {method}. Use 'ibd', 'genomic', or 'pedigree'.")

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()